# Paralelizando Pi Sequencial

In [ ]:
import time
def pi_naive(start, end, step):
  print ("Start: ", str(start))
  print ("End: ", str(end))
  sum = 0.0
  for i in range(start, end):
    x = (i+0.5) * step
    sum = sum + 4.0/(1.0+x*x)
  return sum
if __name__ == "__main__":
  num_steps = 10000000 #10.000.000 (10+e7)
  sums = 0.0
  step = 1.0/num_steps
  tic = time.time() # Tempo Inicial
  sums = pi_naive(0, num_steps, step)
  toc = time.time() # Tempo Final
  pi = step * sums
  print ('Valor de Pi é: {}'.format(pi))
  print ("Tempo Pi: %.8f s" %(toc-tic))

Start:  0
End:  10000000
Valor de Pi é: 3.141592653589731
Tempo Pi: 1.73152232 s


### Usando Process

In [ ]:
import time
from multiprocessing import Process

def pi_naive(start, end, step):
    print("Start: ", str(start))
    print("End: ", str(end))
    sum = 0.0
    for i in range(start, end):
        x =(i+0.5)*step
        sum = sum + 4.0/(1.0+x*x)

    return print (sum*step)

if __name__ == "__main__":
    procs = 4 # numero de processos
    num_steps = 1000000 # numero de steps para o calculo de pi
    sums = 0.0 # sums inicial
    step = 1.0/num_steps  #numero de step
    n = (num_steps + procs-1) // procs #numero de tarefas em relação aos processos 
    tic = time.time() #Tempo Inicial  
    worker = []
    for i in range(procs):
        sums = Process(target=pi_naive, args=(i*n,(i+1)*n,step,)) #iniciando pi
        worker.append(sums)
    for i in range(procs):
        worker[i].start()
    for i in range(procs):
        worker[i].join()
    toc = time.time() #tempo final
    
    #print('Valor Pi: %.10f' %pi)
    print('Tempo Pi: %.8f s' %(toc-tic))

Start:  250000
End:  500000
Start:  0
End:  250000
Start:  500000
End:  750000
Start:  750000
End:  1000000
0.8746757834957989
0.9799146525075235
0.7194139991699205
0.5675882184166324
Tempo Pi: 0.26810455 s


### Usando PIPE

In [ ]:
import time
from multiprocessing import Process, Pipe

def pi_naive(start, end, step,conn):
    print("Start: ", str(start))
    print("End: ", str(end))
    sum = 0.0
    for i in range(start, end):
        x =(i+0.5)*step
        sum = sum + 4.0/(1.0+x*x)
    conn.send(sum) #envia a soma
    conn.close()    
  
if __name__ == "__main__":

    procs = 4 # numero de processos
    num_steps = 1000000 # numero de steps para o calculo de pi
    sums = 0.0 # sums inicial
    step = 1.0/num_steps  #numero de step
    n = (num_steps + procs-1) // procs #numero de tarefas em relação aos processos 
    tic = time.time() #Tempo Inicial
    pi_resultado = []  
    worker = []
    for i in range(procs):
        a, b = Pipe() #inicia o processo Pipe com
        sums = Process(target=pi_naive, args=(i*n,(i+1)*n, step, b)) #iniciando 
        worker.append(sums)
        worker[i].start()
        pi_resultado.append(a.recv()/num_steps)#recebe a soma
        worker[i].join
    toc = time.time() #tempo final
       
    pi_total = 0.0
    for i in range(len(pi_resultado)):
        pi_total = pi_total + pi_resultado[i] #une os resultados
    print('Valor de Pi é: {}'.format(pi_total))
    print('Tempo Pi: %.8f s' %(toc-tic))

Start:  0
End:  250000
Start:  250000
End:  500000
Start:  500000
End:  750000
Start:  750000
End:  1000000
Valor de Pi é: 3.1415926535898753
Tempo Pi: 0.25121140 s


### Usando PIPE + Lock

In [ ]:
import time
from multiprocessing import Process, Pipe, Lock

def pi_naive(start, end, step,conn, lock):
    print("Start: ", str(start))
    print("End: ", str(end))
    sum = 0.0
    for i in range(start, end):
        x =(i+0.5)*step
        sum = sum + 4.0/(1.0+x*x)
    with lock: #para evitar acessos conflitantes na soma final
         conn.send(sum) #envia a soma

if __name__ == "__main__":

    procs = 2 # numero de processos
    num_steps = 1000000 # numero de steps para o calculo de pi
    sums = 0.0 # sums inicial
    step = 1.0/num_steps  #numero de step
    n = (num_steps + procs-1) // procs #numero de tarefas em relação aos processos 
    lock = Lock() # definindo Lock
    tic = time.time() #Tempo Inicial 
    pi_resultado = [] 
    workers = []
    for i in range(procs):
        a, b = Pipe() #inicia o processo Pipe com
        sums = Process(target=pi_naive, args=(i*n,(i+1)*n, step, b, lock,)) #iniciando pi
        workers.append(sums)
        workers[i].start()
        pi_resultado.append(a.recv()/num_steps)#recebe a soma
        workers[i].join
    toc = time.time() #tempo final
       
    pi_total = 0.0
    for i in range(len(pi_resultado)):
        pi_total = pi_total + pi_resultado[i] #une os resultados
    print('Valor de Pi é: {}'.format(pi_total))
    print('Tempo Pi: %.8f s' %(toc-tic))

Start:  0
End:  500000
Start:  500000
End:  1000000
Valor de Pi é: 3.1415926535898993
Tempo Pi: 0.22165036 s


### Usando QUEUE

In [ ]:
import time
from multiprocessing import Process, Queue

def pi_naive(start, end, step, result):
    print("Start: ", str(start))
    print("End: ", str(end))
    sum = 0.0
    for i in range(start, end):
        x =(i+0.5)*step
        sum = sum + 4.0/(1.0+x*x)
    result.put(sum/num_steps) # define o resultado com Queue
if __name__ == "__main__":

    procs = 2 # numero de processos
    Resultado = Queue()
    num_steps = 1000000 # numero de steps para o calculo de pi
    sums = 0.0 # sums inicial
    step = 1.0/num_steps  #numero de step
    n = (num_steps + procs-1) // procs #numero de tarefas em relação aos processos 
    tic = time.time() #Tempo Inicial 
    workers = []
    for i in range(procs):
        worker = Process(target=pi_naive, args=(i*n,(i+1)*n, step, Resultado,)) #iniciando pi
        workers.append(worker)
    for worker in workers:
      worker.start()
    
    for worker in workers:
      worker.join()
    
    toc = time.time() #tempo final
       
    pi = 0.0
    for i in range(procs):
        pi = pi + Resultado.get(i)
    print('Valor de Pi é: {}'.format(pi)) #une os resultados
    print('Tempo Pi: %.8f s' %(toc-tic))

Start:  0
End:  500000
Start:  500000
End:  1000000
Valor de Pi é: 3.1415926535898993
Tempo Pi: 0.21860051 s


### Usando Value

In [ ]:
import time
from multiprocessing import Process, Value

def pi_naive(val,start, end, step):
    print("Start: ", str(start))
    print("End: ", str(end))
    sum = 0.0
    for i in range(start, end):
        x =(i+0.5)*step
        sum = sum + 4.0/(1.0+x*x)
    with val.get_lock():
      val.value += sum

if __name__ == "__main__":

    procs = 2 # numero de processos
    v = Value('d',0,lock=True)
    num_steps = 1000000 # numero de steps para o calculo de pi
    sums = 0.0 # sums inicial
    step = 1.0/num_steps  #numero de step
    n = (num_steps + procs-1) // procs #numero de tarefas em relação aos processos 
    tic = time.time() #Tempo Inicial 
    workers = []
    for i in range(procs):
        worker = Process(target=pi_naive, args=(v,i*n,(i+1)*n, step,)) #iniciando pi
        workers.append(worker)
    for worker in workers:
      worker.start()
    
    for worker in workers:
      worker.join()
    
    toc = time.time() #tempo final
       
    pi = step * v.value
    print('Valor de Pi é: {}'.format(pi)) #une os resultados
    print('Tempo Pi: %.8f s' %(toc-tic))

Start:  0
Start:  500000
End:  1000000
End:  500000
Valor de Pi é: 3.1415926535898993
Tempo Pi: 0.20554328 s


### Usando Pool

In [ ]:
import time
from multiprocessing import Process, Pool

def pi_naive(start, end, step):
    print("Start: ", str(start))
    print("End: ", str(end))
    sum = 0.0
    for i in range(start, end):
        x =(i+0.5)*step
        sum = sum + 4.0 / (1.0 + x * x)
    return sum
           
  
if __name__ == "__main__":

    procs = 2 # numero de processos
    num_steps = 1000000 # numero de steps para o calculo de pi
    sums = 0.0 # sums inicial
    step = 1.0/num_steps  #numero de step
    n = (num_steps + procs-1) // procs #numero de tarefas em relação aos processos 
    p = Pool(processes=procs)
    tic = time.time() #Tempo Inicial

    pi_sums = [p.apply(pi_naive, args=(i*n,(i+1)*n,step,)) for i in range(procs)] #iniciando pi

    toc = time.time() #tempo final

    pi_total = 0.0
    for i in range(len(pi_sums)):
      pi_total = pi_total+pi_sums[i]
    
    print('Valor de Pi é: {}'.format(pi_total * step))
    print('Tempo Pi: %.8f s' %(toc-tic))

Start:  0
End:  500000
Start:  500000
End:  1000000
Valor de Pi é: 3.1415926535898993
Tempo Pi: 0.19571304 s


###Outra forma de se opter o pi paralelo conforme: High Performace Python

In [ ]:
def estimate_nbr_points_in_quarter_cicle(nbr_estimates):
  ""

###Exemplor de POOL
Fonte: https://towardsdatascience.com/how-to-use-the-multiprocessing-package-in-python3-a1c808415ec2

In [ ]:
import time
from multiprocessing import Pool


def cube(x):
    print(f"start process {x}")
    result = x * x * x
    time.sleep(1)
    print(f"end process {x}")
    return result


if __name__ == "__main__":
    ts = time.time()
    pool = Pool(processes=4)
    print([pool.apply(cube, args=(x,)) for x in range(10)])
    pool.close()
    pool.join()
    print("Time in parallel:", time.time() - ts)

start process 0
end process 0
start process 1
end process 1
start process 2
end process 2
start process 3
end process 3
start process 4
end process 4
start process 5
end process 5
start process 6
end process 6
start process 7
end process 7
start process 8
end process 8
start process 9
end process 9
[0, 1, 8, 27, 64, 125, 216, 343, 512, 729]
Time in parallel: 10.160730361938477
